In [1]:
#1. Tính tổng 2 vecto 
def tong_vt(x,y):
    return [i+j for i, j in zip(x, y)]

# tong_vt([1,2],[3,4])

In [107]:
# 2. Tích vô hướng 2 vecto
def tich_vh(x,y):
    kq = 0
    for a, b in zip(x, y):
        kq += a * b
    return kq

# tich_vh([1,2], [3,4])

In [17]:
# 3. Tích ma trận với vecto
def nhan_mt_vt(A, v):
    return [tich_vh(i, v) for i in A]

In [18]:
# 4. Tổng 2 ma trận
def cong_mt(A, B):
    return [[i + j for (i, j) in zip(m,n)] for (m,n) in zip(A,B)]

In [101]:
#5. Tích 2 ma trận 
def tich_mt(A,B):
    kq = [[0 for i in range(len(A))] for j in range(len(B[0]))]
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                kq[i][j] += A[i][k]*B[k][j]
    return kq

In [20]:
#6. Tìm ma trận chuyển vị 
def mt_chuyenvi(A):
    return [[A[i][j] for i in range(len(A))] for j in range(len(A[0]))]

In [25]:
#7. Tìm lũy thừa ma trận
def luythua_mt(A,n):
    if (n <= 1): return A
    return tich_mt(luythua_mt(A, n-1), A)

# A=[[4,3], [2,0]]
# luythua_mt(A, 5)

In [54]:
#8.1. Tìm định thức ma trận
# Hàm xóa dòng và cột
import copy
def mt_del(A, i,j):
    sub_A = copy.deepcopy(A)
    sub_A = sub_A[:i] + sub_A[i+1:]# Bỏ dòng
    # Bỏ cột
    n_row_sub = len(sub_A)
    for i in range(n_row_sub):
        sub_A[i] = sub_A[i][:j] + sub_A[i][j+1:]
    return sub_A 

# Định thức ma trận
def mt_dinhthuc(A):
    if len(A) == 1 and len(A[0]) == 1: # Trường hợp cơ bản, định thức của ma trận 1x1
        return A[0][0]
    total = 0
    for j in range(len(A[0])):# Duyệt qua từng cột để loại bỏ
        sub_A = mt_del(A, 0, j)
        sign = (-1) ** (j % 2)# Tìm dấu
        sub_det = mt_dinhthuc(sub_A)# Gọi đệ quy cho các ma trận con
        total += sign * A[0][j] * sub_det # Cộng dồn định thức khi bỏ cột j
    return total

In [59]:
# bổ sung  thêm Nhân số với ma trận
def NhanMT_So(c, A_list):
    return [[c * a for a in a_row] for a_row in A_list]

#8.2. Tìm ma trận nghịch đảo
def inverse(A):
    n_row = len(A)
    n_col = len(A[0])

    # Tính định thức cho ma trận
    det_A = mt_dinhthuc(A)

    # Trả về None khi ma trận không khả nghịch
    if det_A == 0:
        raise ValueError('Matrix irreversible')
        return None

    # Xử lý trường hợp ma trận cấp 1
    if n_row == 1 and n_col == 1:
        return 1/det_A

    # Tạo ma trận kết quả
    A_res = [[0 for _ in range(n_col)] for _ in range(n_row)]

    # A_res = adj(A)/det_A
    # B1: Chuyển vị ma trận gốc
    A_trans = mt_chuyenvi(A)

    # B2: Tính adj(A) đồng thời gán kết quả vào A_res
    for i_row in range(n_row):
        for i_col in range(n_col):
            # Tạo ra các ma trận con 2x2
            sub_A = mt_del(A_trans, i_row, i_col)

            # Tìm dấu
            sign = (-1) ** (i_row + i_col)

            A_res[i_row][i_col] = sign * mt_dinhthuc(sub_A)

    # B3: Chia định thức
    A_res = NhanMT_So(1/det_A, A_res)
    return A_res

In [56]:
# 9. Các phép biến đổi sơ cấp trên dòng

# Các hàm liên quan: 
def add_vector(v, w):
    return [vi + wi for vi, wi in zip(v, w)]

def mul_scalar_vector(alpha, v):
    return [alpha*vi for vi in v]
# Các phép biến đổi

def row_switch(A, i, k):
    "di <-> dk"
    A[i], A[k] = A[k], A[i]

def row_mul(A, i, alpha):
    "di = anpha*di"
    A[i] = mul_scalar_vector(alpha, A[i])

def row_add(A, i, k, alpha):
    "di = di + anpha*dk"
    A[i] = add_vector(A[i], mul_scalar_vector(alpha, A[k]))

In [57]:
# 10. Tìm nghiệm của hệ phương trình tuyến tính 
# Hàm liên quan: 
import math

def is_zero(x):
    "kiểm tra số thực x có là số thực 0 (rất gần 0)"
    return math.isclose(x, 0, abs_tol=1e-09)
def is_zero_vector(v):
    return all(is_zero(vi) for vi in v)

import sympy
from sympy import Matrix

#10.1. Phép khử Gauss
def Gauss_elimination(A, leading1=True):
    R = A.copy()
    m, n = len(R), len(R[0]) # kích thước ma trận
    row = col = 0
    while row < m:
        # Bước 1
        while col < n and all(is_zero(R[i][col]) for i in range(row, m)):
            col += 1
        if col == n: # đã có dạng bậc thang
            break

        # Bước 2 (chọn dòng đầu tiên có số hạng khác 0)
        pivot_row = row + [not is_zero(R[i][col]) for i in range(row, m)].index(True)
        row_switch(R, row, pivot_row)

        # Bước 3
        if leading1:
            row_mul(R, row, 1/R[row][col])

        # Bước 4
        for i in range(row + 1, m):
            multiplier = R[i][col]/R[row][col]
            row_add(R, i, row, -multiplier)

        # Bước 5
        row += 1
    return R

#10.2 Phép thế ngược
def back_substitution(R):
    "R là ma trận bậc thang của ma trận bổ sung của hệ phương trình Ax = b"
    m, n = len(R), len(R[0]) # kích thước ma trận R

    # vector nghiệm (danh sách theo thứ tự ẩn)
    sol = [None for _ in range(n - 1)]

    # Tìm dòng dưới cùng khác không
    row = m - 1
    while row >= 0 and all(is_zero(R[row][j]) for j in range(n)):
        row -= 1

    if row >= 0 and [not is_zero(R[row][j]) for j in range(n)].index(True) == n - 1:
        return None # vô nghiệm

    last_pcol = n - 1
    while row >= 0:
        pcol = [not is_zero(R[row][j]) for j in range(n)].index(True)
        for i in range(pcol, last_pcol): # các ẩn tự do
            sol[i] = sympy.symbols(f"x{i + 1}")
        sol[pcol] = (R[row][n - 1] - sum(R[row][j]*sol[j] for j in range(pcol + 1, n - 1)))/R[row][pcol]
        last_pcol = pcol
        row -= 1

    # Bổ sung các ẩn tự do từ 0 đến last_pcol - 1
    for i in range(0, last_pcol):
        sol[i] = sympy.symbols(f"x{i + 1}")
    return sol

A = [[1, 3, -2,  0, 2,  0, 0],
     [2, 6, -5, -2, 4, -3, 0],
     [0, 0,  5, 10, 0, 15, 0],
     [2, 6,  0,  8, 4, 18, 0]]
# R=Gauss_elimination(A)
# back_substitution(R)

In [165]:
#11. Tạo ma trận toàn số 0:
def mt_khong(m,n):
    return [[0 for i in range(n)] for j in range(m)]

A = mt_khong(4, 3)
A

[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]

In [164]:
#12. Ma trận đơn vị: 
def mt_donvi(n):
    return [[1 if i == j else 0 for j in range(n)] for i in range(n)]

A = mt_donvi(3)
A

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]

In [163]:
#13. Tạo dãy số ngẫu nhiên gồm n số
import random
def random_list(n):
    A=[random.randrange(0,n+1) for i in range(n)] 
    A.sort()
    return A

L = random_list(10)
L

[0, 0, 1, 4, 6, 6, 7, 9, 10, 10]

In [40]:
#14. Tạo dãy số ngẫu nhiên gồm n số sao cho tổng bằng 100
import random
def sum_to_hundred(n):
    values = [0.0, 100.0] + [random.random() for _ in range(n - 1)]
    values.sort()
    return [values[i+1] - values[i] for i in range(n)]

In [157]:
# Y = O1 + O2.X^2
def getAb(x, y):
    A = [[1, x[i] * x[i]] for i in range(len(x))]
    return A, y

In [162]:
def data_fitting(A, b):
    # O = (A^T.A)^-1.A^T.y
    
    AT = mt_chuyenvi(A)
    
    tmp1 = inverse(tich_mt(AT, A))
    tmp2 = nhan_mt_vt(AT, b)
    kq = nhan_mt_vt(tmp1, tmp2)
        
    return kq

x = [1, 2, 3, 4]
y = [2, 2, 5, 8]

A, b = getAb(x, y)

X = data_fitting(A, b)
X

[1.0232558139534884, 0.430232558139535]

In [161]:
def markov(P, X, n):
    Pn = luythua_mt(P, n)
    kq = nhan_mt_vt(Pn, X)
    return kq

P = [[0.7, 0.1, 0.2],
   [0.2, 0.8, 0.2],
   [0.1, 0.1, 0.6]]

X = [25, 20, 55]

print(markov(P, X, 2))

[32.05, 39.20000000000001, 28.75]


In [167]:
# Calculate the natural log in Python with math.log
import math
print(math.log(math.e))
print(math.log(1))
print(math.log(10))

1.0
0.0
2.302585092994046
